In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlencode, unquote
import requests
import json
import os

In [2]:
#예보가 17시 기준으로 작성되었기에 17시 이후에 코드실행해야함
#17시 이전이라면 today에 어제 날짜입력!
today = '20210606'
tomorrow = '2021-06-07'

In [3]:
#당진 예보 받아오기

ServiceKey = 'trwjiDmekAQ1IK8EF6uLh9GoBnNJ8fBfonv%2Ff8tmgC3GnzGlbKWuGS6S34kIwlLqU%2F26YJrDwt4nh%2Bo4CMj1Jg%3D%3D'
url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst"
queryString = "?" + urlencode({
  "ServiceKey": unquote(ServiceKey),
  "base_date": today,
  "base_time": "1700",
  "nx": '53', 
  "ny": '114',
  "numOfRows": "102",
  "dataType": "JSON"
})
queryURL = url + queryString
response = requests.get(queryURL)
response = json.loads(response.text)

fcst_df = pd.DataFrame()
date = tomorrow
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0
for i, data in enumerate(response['response']['body']['items']['item']):
    if i > 8:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            #print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            #print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            #print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            #print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            #print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3


# 풍속, 풍향 전처리
def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))
def cos_to_angle(x):
    return 180/np.pi*np.arccos(x)+90
def sin_to_angle(x):
    return 180/np.pi*np.arcsin(x)-90
fcst_df['WindDirection_x'] = fcst_df['WindDirection'].map(lambda x : angle_to_cos(x))
fcst_df['WindDirection_y'] = fcst_df['WindDirection'].map(lambda x : angle_to_sin(x))
fcst_df['Wind_x'] = fcst_df['WindDirection_x'] * fcst_df['WindSpeed']
fcst_df['Wind_y'] = fcst_df['WindDirection_y'] * fcst_df['WindSpeed']

#Cloud 전처리

fcst_df_array = fcst_df.values
time = [9,12,15,18]
rate = [13,39,38,10]
a=b=c=d=0
for i in range(len(fcst_df_array)):
  for j in range(4):
    if i == time[j]:
      if fcst_df_array[i][2] == 1:
        a += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 2:
        b += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 3:
        c += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 4:
        d += fcst_df_array[i][2] * rate[j]
if a>=b and a>=c and a>=d:
  cloud = 1
elif b>=c and b>=d:
  cloud = 2
elif c>=d:
  cloud = 3
else:
  cloud = 4
fcst_df['Cloud'] = cloud

# 불필요 열 정리
fcst_df = fcst_df.drop(['WindDirection'],axis=1)
fcst_df = fcst_df.drop(['WindSpeed'],axis=1)
fcst_df = fcst_df.drop(['WindDirection_x'],axis=1)
fcst_df = fcst_df.drop(['WindDirection_y'],axis=1)
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]

fcst_dangjin = fcst_df

In [4]:
response

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20210606',
      'baseTime': '1700',
      'category': 'POP',
      'fcstDate': '20210606',
      'fcstTime': '2100',
      'fcstValue': '0',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210606',
      'baseTime': '1700',
      'category': 'PTY',
      'fcstDate': '20210606',
      'fcstTime': '2100',
      'fcstValue': '0',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210606',
      'baseTime': '1700',
      'category': 'REH',
      'fcstDate': '20210606',
      'fcstTime': '2100',
      'fcstValue': '85',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210606',
      'baseTime': '1700',
      'category': 'SKY',
      'fcstDate': '20210606',
      'fcstTime': '2100',
      'fcstValue': '1',
      'nx': 53,
      'ny': 114},
     {'baseDate': '20210606',
      'baseTime': '1700',
      'category': 'T3H',
      'fcstDa

In [5]:
fcst_dangjin

,Forecast_time,Humidity,Cloud,Temperature,Wind_x,Wind_y
0,2021-06-07 0:00,90.000000,4,18.000000,0.109906,-2.097122
1,2021-06-07 1:00,91.666667,4,18.000000,0.138218,-1.860206
2,2021-06-07 2:00,93.333333,4,18.000000,0.166530,-1.623291
3,2021-06-07 3:00,95.000000,4,18.000000,0.194842,-1.386375
4,2021-06-07 4:00,95.000000,4,18.000000,0.226664,-1.312368
5,2021-06-07 5:00,95.000000,4,18.000000,0.258485,-1.238362
6,2021-06-07 6:00,95.000000,4,18.000000,0.290306,-1.164355
7,2021-06-07 7:00,88.333333,4,19.333333,-0.009827,-2.060231
8,2021-06-07 8:00,81.666667,4,20.666667,-0.309961,-2.956108
9,2021-06-07 9:00,75.000000,4,22.000000,-0.610094,-3.851985


In [6]:
#울산 예보 받아오기

ServiceKey = 'trwjiDmekAQ1IK8EF6uLh9GoBnNJ8fBfonv%2Ff8tmgC3GnzGlbKWuGS6S34kIwlLqU%2F26YJrDwt4nh%2Bo4CMj1Jg%3D%3D'
url = "http://apis.data.go.kr/1360000/VilageFcstInfoService/getVilageFcst"
queryString = "?" + urlencode({
  "ServiceKey": unquote(ServiceKey),
  "base_date": today,
  "base_time": "1700",
  "nx": '102', 
  "ny": '83',
  "numOfRows": "102",
  "dataType": "JSON"
})
queryURL = url + queryString
response = requests.get(queryURL)
response = json.loads(response.text)

fcst_df = pd.DataFrame()
date = tomorrow
fcst_df['Forecast_time'] = [f'{date} {hour}:00' for hour in range(24)]
row_idx = 0
for i, data in enumerate(response['response']['body']['items']['item']):
    if i > 8:
        if data['category']=='REH':
            fcst_df.loc[row_idx, 'Humidity'] = float(data['fcstValue'])
            #print('category:Humidity,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='T3H':
            fcst_df.loc[row_idx, 'Temperature'] = float(data['fcstValue'])
            #print('category:Temperature,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='SKY':
            fcst_df.loc[row_idx, 'Cloud'] = float(data['fcstValue'])
            #print('category:Cloud,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='VEC':
            fcst_df.loc[row_idx, 'WindDirection'] = float(data['fcstValue'])
            #print('category:WindDirection,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'])
        elif data['category']=='WSD':
            fcst_df.loc[row_idx, 'WindSpeed'] = float(data['fcstValue'])
            #print('category:WindSpeed,',data['category'], 'baseTime:',data['baseTime'], ', fcstTime:', data['fcstTime'], ', fcstValue:', data['fcstValue'], '\n')
            row_idx+=3


# 풍속, 풍향 전처리
def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))
def cos_to_angle(x):
    return 180/np.pi*np.arccos(x)+90
def sin_to_angle(x):
    return 180/np.pi*np.arcsin(x)-90
fcst_df['WindDirection_x'] = fcst_df['WindDirection'].map(lambda x : angle_to_cos(x))
fcst_df['WindDirection_y'] = fcst_df['WindDirection'].map(lambda x : angle_to_sin(x))
fcst_df['Wind_x'] = fcst_df['WindDirection_x'] * fcst_df['WindSpeed']
fcst_df['Wind_y'] = fcst_df['WindDirection_y'] * fcst_df['WindSpeed']

#Cloud 전처리

fcst_df_array = fcst_df.values
time = [9,12,15,18]
rate = [13,39,38,10]
a=b=c=d=0
for i in range(len(fcst_df_array)):
  for j in range(4):
    if i == time[j]:
      if fcst_df_array[i][2] == 1:
        a += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 2:
        b += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 3:
        c += fcst_df_array[i][2] * rate[j]
      if fcst_df_array[i][2] == 4:
        d += fcst_df_array[i][2] * rate[j]
if a>=b and a>=c and a>=d:
  cloud = 1
elif b>=c and b>=d:
  cloud = 2
elif c>=d:
  cloud = 3
else:
  cloud = 4
fcst_df['Cloud'] = cloud

# 불필요 열 정리
fcst_df = fcst_df.drop(['WindDirection'],axis=1)
fcst_df = fcst_df.drop(['WindSpeed'],axis=1)
fcst_df = fcst_df.drop(['WindDirection_x'],axis=1)
fcst_df = fcst_df.drop(['WindDirection_y'],axis=1)
fcst_df = fcst_df.interpolate()
fcst_df = fcst_df.iloc[:24]

fcst_ulsan = fcst_df

In [7]:
fcst_ulsan

,Forecast_time,Humidity,Cloud,Temperature,Wind_x,Wind_y
0,2021-06-07 0:00,80.000000,1,21.000000,-1.400000,-3.429011e-16
1,2021-06-07 1:00,81.666667,1,20.333333,-1.261686,-4.202724e-01
2,2021-06-07 2:00,83.333333,1,19.666667,-1.123372,-8.405448e-01
3,2021-06-07 3:00,85.000000,1,19.000000,-0.985058,-1.260817e+00
4,2021-06-07 4:00,85.000000,1,18.666667,-1.189129,-1.241755e+00
5,2021-06-07 5:00,85.000000,1,18.333333,-1.393200,-1.222692e+00
6,2021-06-07 6:00,85.000000,1,18.000000,-1.597271,-1.203630e+00
7,2021-06-07 7:00,75.000000,1,20.666667,-1.682911,-1.131050e+00
8,2021-06-07 8:00,65.000000,1,23.333333,-1.768550,-1.058470e+00
9,2021-06-07 9:00,55.000000,1,26.000000,-1.854190,-9.858903e-01
